In [ ]:
HTML(read(open("style.html"), String))

In [1]:
include("silent.jl")

@silent (macro with 1 method)

In [2]:
using Pkg
@silent Pkg.add("NBInclude")
using NBInclude
import Chess
import Chess.UCI
import Chess.PGN
@silent Pkg.add("Plots")
import Plots

In [3]:
@nbinclude "2.1 - Board.ipynb"
@nbinclude "2.0 - Gameplay.ipynb"
@nbinclude "3.1 - Evaluation.ipynb"
@nbinclude "5.1 - Alpha-Beta-Pruning AI.ipynb"
@nbinclude "6.0 - Iterative Deepening.ipynb"

getNextMove (generic function with 6 methods)

***

# Testing - Stockfish

Dieses Notebook umfasst Tests gegen die Schach-Engine [`Stockfish 15.1`](https://stockfishchess.org/).

Stockfish ist mit dem [Universal Chess Interface (UCI)](https://www.chessprogramming.org/UCI) Kommunikationsprotokoll kompatibel. Dies ermöglicht es uns, Stockfish über das UCI-Modul der `Chess.jl` Bibliothek einzubinden.

Da es sich bei Stockfish um eine hochgradig optimierte Schach-KI mit einer geschätzten Elo von über 3500 handelt, muss Stockfish für unsere Testzwecke geschwächt werden, um einen gleichwertigen Gegner darstellen zu können.

Die Spielstärke von Stockfish wird unter anderem durch zwei Einstellungen bestimmt:

### Skill Level

Mit der UCI-Option `Skill Level` lässt sich eine Spielstärke `0 - 19` angeben. Bei einem Skill-Level von 19 erricht Stockfish seine höchste Spielleistung.

### UCI Elo Rating
    
Mit den UCI-Optionen `UCI_LimitStrength` und `UCI_Elo` lässt sich eine ungefähre Spielstärke in Elo-Punkten angeben. Diese Limitierung wird [intern](https://github.com/official-stockfish/Stockfish/blob/d99942f25449789de78c9d36e3dcb67d4eb04e98/src/search.cpp#L93) wiefolgt in ein passendes Skill Level konvertiert:

```c++
    double e = double(uci_elo - 1320) / (3190 - 1320);
    level = std::clamp((((37.2473 * e - 40.8525) * e + 22.2943) * e - 0.311438), 0.0, 19.0);
```

Quelle: [github.com/official-stockfish/Stockfish](https://github.com/official-stockfish/Stockfish/blob/d99942f25449789de78c9d36e3dcb67d4eb04e98/src/search.cpp#L101), Abruf am 04.06.2023.

Eine Anzahl von 1500 Elo-Punkten entspricht somit einem Skill-Level von 1,48924. Wie im Source-Code beschrieben, handelt es sich hierbei jedoch lediglich um Annäherungen.

Aus der beschrieben Formel ergibt sich die folgende Skill-Level Interpretation:

<table style="width: 70%">
    <tr>
        <th>Skill-Level</th>
        <th>~Elo</th>
    </tr>
    <tr><td>0</td><td>≥ 1320</td></tr>
    <tr><td>1</td><td>≥ 1445</td></tr>
    <tr><td>2</td><td>≥ 1567</td></tr>
    <tr><td>3</td><td>≥ 1729</td></tr>
    <tr><td>4</td><td>≥ 1954</td></tr>
    <tr><td>5</td><td>≥ 2197</td></tr>
    <tr><td>6</td><td>≥ 2383</td></tr>
    <tr><td>7</td><td>≥ 2519</td></tr>
    <tr><td>8</td><td>≥ 2625</td></tr>
    <tr><td>9</td><td>≥ 2712</td></tr>
    <tr><td>10</td><td>≥ 2786</td></tr>
    <tr><td>11</td><td>≥ 2852</td></tr>
    <tr><td>12</td><td>≥ 2910</td></tr>
    <tr><td>13</td><td>≥ 2963</td></tr>
    <tr><td>14</td><td>≥ 3012</td></tr>
    <tr><td>15</td><td>≥ 3057</td></tr>
    <tr><td>16</td><td>≥ 3100</td></tr>
    <tr><td>17</td><td>≥ 3139</td></tr>
    <tr><td>18</td><td>≥ 3177</td></tr>
    <tr><td>19</td><td>≥ 3212</td></tr>
</table>

Um die gewünschte Mindeststärke von 1300 zu erreichen, muss unsere KI Stockfish mit Level 0 bei gleicher Suchtiefe in mindestens 50% der Fälle schlagen (ausgenommen Unentschieden).

# Stockfish KI

Um Stockfish als Spieler einbinden zu können, wird ein eigener KI-Typ definiert, welcher über das `Chess.UCI` Modul auf Stockfish zugreift.

Der Konstruktor `Stockfish(skill::Int, depth::Int = 20)` erstellt eine neue Instanz des Stockfish-Spielers mit einer Skill-Level- und Suchtiefen-Limitierungen (optional).

Der Konstruktor `Stockfish(elo::Int)` erstellt eine neue Instanz des Stockfish-Spielers mit einer Elo-Limitierung. Hierbei wird Stockfish auf eine feste Suchtiefe von 20 (Halbzügen) gesetzt. Dies ermöglicht es Stockfish, gute Zugentscheidungen zu treffen, ohne einen unfairen Vorteil zu erlangen.

In [4]:
struct Stockfish
    engine::Chess.UCI.Engine
    depth::Int
    
    function Stockfish(skill::Int, depth::Int = 20)
        # requires the stockfish executable (f.e. stockfish.exe) to be in PATH
        sf = Chess.UCI.runengine("stockfish")
        Chess.UCI.setoption(sf, "Skill Level", skill)
        return new(sf, depth) 
    end
    
    function Stockfish(elo::Int)
        # requires the stockfish executable (f.e. stockfish.exe) to be in PATH
        sf = Chess.UCI.runengine("stockfish")
        Chess.UCI.setoption(sf, "UCI_LimitStrength", true)
        Chess.UCI.setoption(sf, "UCI_Elo", elo)
        return new(sf, 20) 
    end
end

getNextMove (generic function with 7 methods)

**Input:**
+ ai &rarr; die Stockfish-KI
+ extboard &rarr; das Spielbrett
+ log &rarr; die zu verwendende Log-Funktion
+ _... &rarr; ein Platzhalter, der es ermöglicht weitere Parameter mitzugeben 

**Output:**
+ move &rarr; der von Stockfish gefundene Zug

In [ ]:
function getNextMove(ai::Stockfish, extboard::ExtendedBoard, log::Function, _...)::Chess.Move
    Chess.UCI.setboard(ai.engine, extboard.board)
    return Chess.UCI.search(ai.engine, "go depth $(ai.depth)").bestmove
end

# Aufstellung der Tests

Tests gegen Stockfish werden mit dem folgenden Code durchgeführt. Hierbei tritt die beste Version unserer KI (ABMemoAllAI) mit einer maximalen Suchzeit von 10 Sekunden gegen Stockfish an.

Während diese Zeitlimitierung die Spielstärke unserer KI negativ beeinflusst, ermöglicht sie uns, möglichst viele Spiele zu spielen.

Alle Tests werden mehrfach wiederholt, nach jedem Spiel werden die Farben getauscht.

Gespielte Spiele werden anschließend in entsprechenden Ordnern `./games-$(elo)`, bzw. `./games-$(level)-$(depth)` gespeichert.

**Tests mit Stockfish Elo Limitierung**

In [ ]:
# --- parameter ---
n = 100
elo = 1500
# ---
for x in 1:n
    sf = Stockfish(elo)
    ai = IterativeDeepening(
        ABMemoAllAI, # backing AI Type
        1000,        # max Depth
        10.0,        # max Time in Seconds
        """          # required dependencies for backing AI Type
            using NBInclude
            @nbinclude "3.1 - Evaluation.ipynb"
            @nbinclude "5.1 - Alpha-Beta-Pruning AI.ipynb"
        """
    )
    if x % 2 == 0
        play(white = ai, black = sf, save = "./games-$(elo)")
    else
        play(black = ai, white = sf, save = "./games-$(elo)") 
    end
    cleanup!(ai)
    Chess.UCI.quit(sf.engine)
end

r2q1rk1/ppp1bppp/2n5/4p3/7P/1NP3P1/PP2QP2/R1B1KBNb b Q -


HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/br.svg\" y=\"0\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wr.svg\" y=\"7\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wn.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"1\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bp.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bn.svg\" y=\"2\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"7\" class=\"dark-square\" height=\"1\"

Last Move: Move(h3h4)
Killed worker 733 after 735.1758259 seconds


**Tests mit Stockfish Skill-Level Limitierung**

In [ ]:
# --- parameter ---
n = 100
level = 3
depth = 5
# ---
for x in 1:n
    sf = Stockfish(level, depth)
    ai = IterativeDeepening(
        ABMemoAllAI, # backing AI Type
        1000,        # max Depth
        10.0,        # max Time in Seconds
        """          # required dependencies for backing AI Type
            using NBInclude
            @nbinclude "3.1 - Evaluation.ipynb"
            @nbinclude "5.1 - Alpha-Beta-Pruning AI.ipynb"
        """
    )
    if x % 2 == 0
        play(white = ai, black = sf, save = "./games-$(level)-$(depth)")
    else
        play(black = ai, white = sf, save = "./games-$(level)-$(depth)")
    end
    cleanup!(ai)
    Chess.UCI.quit(sf.engine)
end

Um zu gewährleisten, dass es sich bei den gespielten Testspielen um einzigartige Spiele handelt, prüfen wir alle Spiele unter einer gegebenen Konfiguration auf Duplikate. Das Vorkommen eines Duplikates invalidiert keinesfalls die erspielten Ergebnisse, dennoch sollte die Anzahl aller Duplikate gering ausfallen, um ein zuverlässiges Ergebnis zu liefern.

In [ ]:
for gameSet in readdir()
    if !startswith(gameSet, "games-")
        continue
    end
    println("validating $gameSet")
    knownGames = []
    for game in readdir(gameSet)
        if !endswith(game, ".pgn")
            continue
        end
        pgn = read(gameSet * "/" * game, String)
        if pgn in knownGames
            println("found a duplicate: $gameSet/$game")
        else
            push!(knownGames, pgn)
        end
    end
    println("checked $(length(knownGames)) games")
end

Unter einer gegebenen Konfiguration bestimmen wir die Siegrate mit der Funktion `getWinRate`.

**Input:**
+ folder &rarr; der Ordner mit allen Spielen, z.B. "games-1500" oder "games-1-3"

**Output:**
+ die Siegrate unserer KI, Wert zwischen 0.0 und 1.0

In [ ]:
function getWinRate(folder::String)::Float64
    results = Dict(:SF_White => 0, :SF_Black => 0, :AI_White => 0, :AI_Black => 0, :Draw => 0)
    index = 1
    for file in readdir(folder, sort = true)
        stream = open("$(folder)/$(file)", "r")
        game = Chess.PGN.readgame(Chess.PGN.PGNReader(stream))
        close(stream)
        board = last(Chess.boards(game))
        if Chess.isdraw(board)
            results[:Draw] += 1
        elseif Chess.sidetomove(board) == Chess.WHITE
            if index % 2 == 1
                results[:AI_Black] += 1
            else
                results[:SF_Black] += 1
            end
        else
            if index % 2 == 1
                results[:SF_White] += 1
            else
                results[:AI_White] += 1
            end
        end
        index += 1
    end
    totalNoDrawsAI = results[:AI_White] + results[:AI_Black]
    totalNoDrawsSF = results[:SF_White] + results[:SF_Black]
    totalNoDraws = totalNoDrawsAI + totalNoDrawsSF
    println("$(folder):")
    println("\t$(results)")
    println("\tWins:    $(round(((results[:AI_White] + results[:AI_Black]) / totalNoDraws) * 100, digits = 2)
        )% (excluding draws, of which $(round((results[:AI_White] / totalNoDrawsAI) * 100, digits = 2)
        )% as White, $(round((results[:AI_Black] / totalNoDrawsAI) * 100, digits = 2))% as Black)")
    println("\tDefeats: $(round(((results[:SF_White] + results[:SF_Black]) / totalNoDraws) * 100, digits = 2)
        )% (excluding draws, of which $(round((results[:SF_Black] / totalNoDrawsSF) * 100, digits = 2)
        )% as White, $(round((results[:SF_White] / totalNoDrawsSF) * 100, digits = 2))% as Black)")
    println("\tDraws:   $(round((results[:Draw] / (total + results[:Draw])) * 100, digits = 2))% (all games)")
    return (results[:AI_White] + results[:AI_Black]) / totalNoDraws
end

Aus den Konfigurationen Skill-Level + Suchtiefe ergibt sich der folgende Graph:

In [ ]:
gameSets = []
for gameSet in readdir()
    if match(r"games-\d+-\d+", gameSet) == nothing
        continue
    end
    push!(gameSets, gameSet)
end
gameSets = sort(gameSets, by=name -> parse(Int, split(name, "-")[2]) * 100 + parse(Int, split(name, "-")[3]))

winrates = [getWinRate(gameSet) * 100 for gameSet in gameSets]
configs = [gameSet[7:length(gameSet)] for gameSet in gameSets]
Plots.plot(configs, winrates, label = "Siegrate (%)")

Unsere KI ist bei den Stockfish Leveln 1 (~1445 Elo) und 2 (~1567 Elo) mit einer Suchtiefe von 20 in der Lage, eine Siegrate > 50% zu erzielen. Daraus ist zu folgern, dass die Elo unserer KI über 1300 Punkten liegt.

In [ ]:
for x in 1:10
    elo = 1500
    sf = Stockfish(elo)
    ai = IterativeDeepening(
        ABMemoAllAI, # backing AI Type
        100,         # max Depth
        10.0,        # max Time in Seconds
        """          # required dependencies for backing AI Type
            using NBInclude
            @nbinclude "3.1 - Evaluation.ipynb"
            @nbinclude "5.1 - Alpha-Beta-Pruning AI.ipynb"
        """
    )
    if x % 2 == 0
        play(white = ai, black = sf, save = "./games-$(elo)")
    else
        play(black = ai, white = sf, save = "./games-$(elo)") 
    end
    cleanup!(ai)
    Chess.UCI.quit(sf.engine)
end

8/2b3k1/8/5bNp/8/2K4P/P1P5/8 w - -


HTML{String}("Board:<div class=\"chessboard\"><style soped=\"true\">.dark-square {\n    fill: #c3936a;\n    stroke-width: 0;\n}\n.light-square {\n    fill: #ffd7a6;\n    stroke-width: 0;\n}\n.highlight {\n    fill: #47d18b;\n}\n@media (prefers-color-scheme: dark) {\n    .dark-square {\n        fill: #00796b;\n    }\n    .light-square {\n        fill: #3ca59b;\n    }\n    .highlight {\n        fill: #d23600;\n    }\n}\n</style><svg style=\"float: left; margin-right: 20px\" viewBox=\"0 0 8 8\" height=\"280\" width=\"280\"><g><g><rect x=\"0\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"0\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"0\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"0\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"1\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"0\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"1\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/bb.svg\" y=\"1\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"2\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"3\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"4\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"2\" y=\"5\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wk.svg\" y=\"5\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"6\" class=\"light-square\" height=\"1\" width=\"1\"></rect><image x=\"2\" xlink:href=\"https://raw.githubusercontent.com/romstad/Chess.jl/master/img/wp.svg\" y=\"6\" height=\"1\" width=\"1\"></image></g><g><rect x=\"2\" y=\"7\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"0\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"1\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"2\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"3\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"4\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"5\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"6\" class=\"dark-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"3\" y=\"7\" class=\"light-square\" height=\"1\" width=\"1\"></rect><g></g></g><g><rect x=\"4\" y=\"0\" class=\"light-squa

Last Move: Move(c8f5)
Spawned new worker 749 in 20.5429362 seconds
Active worker found: 749
Starting with depth 1
